# Исследование методов построения рекомендательных систем в области продажи товаров через интернет
## Введение
На данный момент многие сервисы по продаже товаров через Интернет имеют рекомендатеции. Для этого соответственно необходимо разработать рекомендательную систему. Алгоритмы рекомендательных систем обычно разделяют на два типа: основанные на контенте и выборки коллаборативной фильтрации. Но в современных рекомендательных системах используют оба этих алгоритма.

## Алгоритм основанный на контенте
Данный алгоритм основывается, как можно понять из названия, на контенте, который более предпочтителен пользователю. Главная задача которая стоит перед разработчиками рекомендательных систем - дать возможность пользователю указывать какой контент ему более предпочтителен, а какой ему не интересен.
## Алгоритм коллаборативной фильтрации
Данный алгоритм основывается на том, что система предлагает пользователю контент, на основе преддпочтений пользователей со смежными интересами. Для этого применяется матричная факторизация где в столбцах располагаются товары, а строки состоят из пользователей и в каждой ячейке располагается известные оценки пользователей. Главной задачей матричной факторизацией является заполнение пропусков в данной таблице.

## Подготовка среды разработки
Перед началом работы с библиотеками машинного обучения ML.Net и визуализации XPlot. Необходимо установить nuget-пакеты и подключить данные библиотеки. Команды ниже позволяют это сделать. 

In [ ]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML"
    
//Install XPlot package
#r "nuget:XPlot.Plotly"

//Install Recommender
#r "nuget:Microsoft.ML.Recommender"

using Microsoft.ML;
using Microsoft.ML.Recommender;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using System.Linq;
using System.Text.Json;
using System.IO;
using XPlot.Plotly;

Installed Packages Microsoft.ML, 1.7.0 Microsoft.ML.Recommender, 0.19.0 XPlot.Plotly, 4.0.6

## Подготовка моделей
Датасет для создания рекомендательной системы мы возьмем от компании amazon за 2016 год. Данный датасет содержит модели в которых есть:
1. Идентификатор пользователя
2. Идентификатор товара
3. Массив оценок насколько данный отзыв был полезен
4. Описание отзыва
5. Общая оценка товара
6. Краткое описание отзыва
7. Универсальное время отзыва
8. Время отзыва
В качестве результата у нас будет класс PredictionModel, в котором содержится только итоговая оценка и идентификатор товара

In [ ]:
public class ReviewModel
{
    public string reviewerID { get; set; }
    public string asin { get; set; }
    public string reviewerName { get; set; }
    public int[] helpful { get; set; }
    public string reviewText { get; set; }
    /// <summary>
    /// Оценка
    /// </summary>
    public float overall { get; set; }
    public string summary { get; set; }
    public int unixReviewTime { get; set; }
    public string reviewTime { get; set; }
}
public class ProductPrediction
{
    public string asin;
    public float Score;
}

## Загрузка и обработка данных
Раздел который был взят для анализа это датасет, содержащий отзывы об одежде, обуви и ювелирных украшениях. Изначально датасет представляет собой неформатизированный json файл, который надо обработать, чтобы можно было спарсить информацию из него. Далее мы разделяем данные на тестовые и тренировочные. Одни для обучения моделей, а вторые соответственно, чтобы проверить правильность работы системы.

In [ ]:
var fileName = "./Data/Clothing_Shoes_and_Jewelry_5.json";
var jsonString = File.ReadAllText(fileName);
jsonString = jsonString.Replace('\n', ',');
var reviewsSrc = Newtonsoft.Json.JsonConvert.DeserializeObject<List<ReviewModel>>("["+jsonString+"]");
MLContext mlContext = new MLContext();
int countTest = reviewsSrc.Count()/5;
var trainDataView = mlContext.Data.LoadFromEnumerable<ReviewModel>(reviewsSrc);
var testDataView = mlContext.Data.LoadFromEnumerable<ReviewModel>(reviewsSrc.Take(countTest));
display("Schema of training DataView:");
display(trainDataView.Preview(1).RowView);
display(reviewsSrc.Skip(countTest).ToList().Count);
display(reviewsSrc.Count);

Schema of training DataView:

index,Values
0,"[ reviewerID: A1KLRMWW2FWPL4, asin: 0000031887, reviewerName: Amazon Customer ""cameramom"", helpful: { Dense vector of size 2: IsDense: True, Length: 2 }, reviewText: This is a great tutu and at a really great price. It doesn't look cheap at all. I'm so glad I looked on Amazon and found such an affordable tutu that isn't made poorly. A++, overall: 5, summary: Great tutu- not cheaply made, unixReviewTime: 1297468800, reviewTime: 02 12, 2011 ]"


222942

278677

# Анализ данных
Для анализа данных мы будем использовать массив оценок, времен, отзывов, имен пользователей.

Ниже представлено получение столбцов, которые далее будут использованы для диаграмм.

In [ ]:
//Extract some data into arrays for plotting:

int numberOfRows = 1000;
float[] scores = trainDataView.GetColumn<float>(nameof(ReviewModel.overall)).Take(numberOfRows).ToArray();
int[] times = trainDataView.GetColumn<int>(nameof(ReviewModel.unixReviewTime)).Take(numberOfRows).ToArray();
string[] reviews = trainDataView.GetColumn<string>(nameof(ReviewModel.reviewText)).Take(numberOfRows).ToArray();
string[] names = trainDataView.GetColumn<string>(nameof(ReviewModel.reviewerName)).Take(numberOfRows).ToArray();

Рассмотрим соотношение количество оценок и их значение. По оси абсцисс будут перечислены оценки, а по оси ординат их количество. Как мы видим на рисунке ниже наибольшее количество оценок на Амазоне составляют пятерки, и далее идет на уменьшение вплоть до 1. Это показывает то, что большинство пользователей оставляют положительные оценки и необходимо устанавливать минимальную оценку для рекомендации товара на отметке 3.75 .

In [ ]:
// Distribution of number of scores
//XPlot Histogram reference: http://tpetricek.github.io/XPlot/reference/xplot-plotly-graph-histogram.html

var faresHistogram = Chart.Plot(new Histogram(){x = scores, autobinx = false, nbinsx = 20});
var layout = new Layout.Layout(){title="Количество оценок"};
faresHistogram.WithLayout(layout);
faresHistogram.WithXTitle("Оценки");
faresHistogram.WithYTitle("Количество");
faresHistogram.Show();
display(faresHistogram);

Height,Id,PlotlySrc,Width
500,ef319e13-b7b4-471c-b1c9-61ab9e172752,https://cdn.plot.ly/plotly-latest.min.js,900


Рассмотрим зависимость оценок от длины их отзыва, это нам позволит узнать кто оставляет большие отзывы, те кто недовольны или те кого всё устраивает. Для начала выведем plot с помощью которого можно будет определить выбросы, чтобы в дальнейшем отбросить эти значения и построить информативный boxplot. Рассмотрим гистограмму ниже, на ней видно, что основная часть отзывов варьируется в диапозоне от 0 до 2000 символов и есть некоторые выбросы выше вплоть до 10000. При построении boxplot`а данные значение не будут браться в расчет.

In [ ]:
var chartFareVsTime = Chart.Plot(
    new Scatter()
    {
        x =scores.Take(200) ,
        y = reviews.Select(x=>x.Length).Take(200).ToArray(),
        mode = "markers",
        marker = new Marker()
        {
            color = scores,
            colorscale = "Jet"
        }
    }
);

var layout = new Layout.Layout(){title="Plot с отношениями длин отзывов к их оценкам"};
chartFareVsTime.WithLayout(layout);
chartFareVsTime.Width = 500;
chartFareVsTime.Height = 500;
chartFareVsTime.WithXTitle("Оценки");
chartFareVsTime.WithYTitle("Длина оценки");
chartFareVsTime.WithLegend(false);
chartFareVsTime.Show();
display(chartFareVsTime);

Height,Id,PlotlySrc,Width
500,6233ac9d-6fe7-40b8-83ea-ff20bce0dcb6,https://cdn.plot.ly/plotly-latest.min.js,500


Рассмотрим зависимость длины отзыва от его оценки, для этого используем вид гистограммы boxplot. Данная гистограмма позволяет определить в каком диапозоне находится основная часть значений и определить выбросы. Такж не стоит забывать об установленном значении в 2000 символов больше которых мы не берем в расчет. Рассмотрим данный boxplot, хоть мы и видим что у положительных отзывов гораздо больше оценок с длиной отзывов близким к 2000, но основные значения у всех оценок находятся на одном уровне. Это означает, что так как положительных оценок больше, то и в общем будет гораздо больше отзывов и по статистике будет больше отзывов с большей длиной отзывов, но основная масса также находится на отметке в 200-400 символов, также как и у остальных оценок.

In [ ]:
var chartFareVsTime = Chart.Plot(
    new Box()
    {
        x =scores ,
        y = reviews.Select(x=>x.Length).Where(x=>x<2000).ToArray()
    }
);

var layout = new Layout.Layout(){title="Boxplot с отношениями длин отзывов к их оценкам"};
chartFareVsTime.WithLayout(layout);
chartFareVsTime.Width = 500;
chartFareVsTime.Height = 500;
chartFareVsTime.WithXTitle("Оценки");
chartFareVsTime.WithYTitle("Длина отзыва");
chartFareVsTime.WithLegend(false);
chartFareVsTime.Show();
display(chartFareVsTime);

Height,Id,PlotlySrc,Width
500,8fc7bddd-cce8-46d5-88d4-b598f5c9d528,https://cdn.plot.ly/plotly-latest.min.js,500


## Создание и обучение модели машинного обучения
Для создания модели машинного обучения при матричной факторизации необходимо определить идентификатор пользователя и идентификатор товара. Затем добавить приведенные к целочисленным значениям идентификаторы и задать настройки матричной факторизации для естимейтора. Мы будем использовать метод Recommendation. Который возвращает обучающие модели и у них выберем матричную факторизацию.

In [ ]:
IEstimator<ITransformer> estimator = mlContext
    .Transforms
    .Conversion
    .MapValueToKey(outputColumnName: "ReviewerIdEncoded", inputColumnName: $"{nameof(ReviewModel.reviewerID)}")
    .Append(mlContext
        .Transforms
        .Conversion
        .MapValueToKey(outputColumnName: "ProductIdEncoded", inputColumnName: $"{nameof(ReviewModel.asin)}"));
var options = new MatrixFactorizationTrainer.Options
{
    MatrixColumnIndexColumnName = "ReviewerIdEncoded",
    MatrixRowIndexColumnName = "ProductIdEncoded",
    LabelColumnName = $"{nameof(ReviewModel.overall)}",
    NumberOfIterations = 20,
    ApproximationRank = 100
};
var trainerEstimator = estimator
    .Append(mlContext
        .Recommendation()
        .Trainers
        .MatrixFactorization(options));
Console.WriteLine("========================== Training the model =============================");
ITransformer model = trainerEstimator.Fit(trainDataView);
display(model.GetOutputSchema(trainDataView.Schema))

========================== Training the model =============================


index Name Index IsHidden Type Annotations 0 reviewerID 0 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 1 asin 1 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 2 reviewerName 2 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 3 helpful 3 False Dimensions IsKnownSize ItemType Size RawType [ 0 ] False { Int32: RawType: System.Int32 } 0 Microsoft.ML.Data.VBuffer<System.Int32> Schema [ ] 4 reviewText 4 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 5 overall 5 False RawType System.Single Schema [ ] 6 summary 6 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 7 unixReviewTime 7 False RawType System.Int32 Schema [ ] 8 reviewTime 8 False RawType System.ReadOnlyMemory<System.Char> Schema [ ] 9 ReviewerIdEncoded 9 False Count RawType 39387 System.UInt32 Schema [ { KeyValues: Vector<String, 39387>: Name: KeyValues, Index: 0, IsHidden: False, Type: { Vector<String, 39387>: Dimensions: [ 39387 ], IsKnownSize: True, ItemType: { String: RawType: System.ReadOnlyMemory<System.Char> }, Size: 39387, RawType: Microsoft.ML.Data.VBuffer<System.ReadOnlyMemory<System.Char>> }, Annotations: { : Schema: [ ] } } ] 10 ProductIdEncoded 10 False Count RawType 23033 System.UInt32 Schema [ { KeyValues: Vector<String, 23033>: Name: KeyValues, Index: 0, IsHidden: False, Type: { Vector<String, 23033>: Dimensions: [ 23033 ], IsKnownSize: True, ItemType: { String: RawType: System.ReadOnlyMemory<System.Char> }, Size: 23033, RawType: Microsoft.ML.Data.VBuffer<System.ReadOnlyMemory<System.Char>> }, Annotations: { : Schema: [ ] } } ] 11 Score 11 False RawType System.Single Schema [ { ScoreColumnSetId: Key<UInt32, 0-2147483646>: Name: ScoreColumnSetId, Index: 0, IsHidden: False, Type: { Key<UInt32, 0-2147483646>: Count: 2147483647, RawType: System.UInt32 }, Annotations: { : Schema: [ ] } }, { ScoreColumnKind: String: Name: ScoreColumnKind, Index: 1, IsHidden: False, Type: { String: RawType: System.ReadOnlyMemory<System.Char> }, Annotations: { : Schema: [ ] } }, { ScoreValueKind: String: Name: ScoreValueKind, Index: 2, IsHidden: False, Type: { String: RawType: System.ReadOnlyMemory<System.Char> }, Annotations: { : Schema: [ ] } } ]

## Оценка модели
Оценим обученную модель с помощью тестовых данных, которые мы определили в самом начале. На выходе мы получаем значения ошибки и квадратичное-R, которое означает насколько модель верна. В результате мы получаем неплохие значения, где r-squared - 0.89 и квадратичное значение ошибки - 0.36. Данные значения в рамках нормы и мы считаем, что модель обучена хорошо.

In [ ]:
public static void EvaluateModel(MLContext mlContext, IDataView testDataView, ITransformer model)
{
    Console.WriteLine("========================== Evaluating the model =============================");
    var prediction = model.Transform(testDataView);
    var metrics = mlContext
        .Regression
        .Evaluate(prediction, labelColumnName: $"{nameof(ReviewModel.overall)}", scoreColumnName: $"{nameof(ProductPrediction.Score)}");
    Console.WriteLine("Root Mean Squared Error : " + metrics.RootMeanSquaredError.ToString());
    Console.WriteLine("RSquared: " + metrics.RSquared.ToString());
}
EvaluateModel(mlContext, testDataView, model);

========================== Evaluating the model =============================
Root Mean Squared Error : 0.3637768376873756
RSquared: 0.8870391363163995


## Предсказания модели
Рассмотрим как обученная модель будет предсказывать оценки пользователя. На вход мы будем подавать товар и пользователя, а обученная модель будет предполагать, какую бы оценку поставил пользователь и если данное предположение больше 3.75 баллов, то значит товар можно будет рекомендовать. Перед экспериментом мы нашли пользователя, который оставлял только положительные отзывы и мы можем предположить, что это пользователь, который остается доволен продукцией магазина. При анализе 10 товаров было выявлено, что ему могут рекомендоваться все эти товары. Но мы также можем использовать не установление жесткого ограничения, как мы сделали ранее установив границу для рекомендации товара в 3.75. К примеру можно сортировать по убыванию по оценке и первые три товара брать как рекомендованные.

In [ ]:
public static void UseModelForSinglePrediction(MLContext mlContext, ITransformer model, IEnumerable<ReviewModel> products)
{
    Console.WriteLine("=========================== Making a prediction =============================");
    var predictionEngine = mlContext
        .Model
        .CreatePredictionEngine<ReviewModel, ProductPrediction>(model);

    foreach(var product in products.GroupBy(x=>x.asin).Select(x=>x.Key).Take(10))
    {
        var testInput = new ReviewModel { reviewerID = "A1KLRMWW2FWPL4", asin = product};
        var movieRatingPrediction = predictionEngine.Predict(testInput);
        if (Math.Round(movieRatingPrediction.Score, 1) > 3.75)
        {
            Console.WriteLine("Товар " + testInput.asin + " рекомендуется пользователю " + testInput.reviewerID + ". Общая оценка =" + movieRatingPrediction.Score + ".");
        }
        else
        {
            Console.WriteLine("Товар " + testInput.asin + " не рекомендуется пользователю " + testInput.reviewerID + ". Общая оценка =" + movieRatingPrediction.Score + ".");
        }
    }
}
UseModelForSinglePrediction(mlContext, model, reviewsSrc);

=========================== Making a prediction =============================
Product 0000031887 is recommended for user A1KLRMWW2FWPL4. Score=4.7728386
Product 0123456479 is recommended for user A1KLRMWW2FWPL4. Score=4.377736
Product 1608299953 is recommended for user A1KLRMWW2FWPL4. Score=3.968997
Product 1617160377 is recommended for user A1KLRMWW2FWPL4. Score=4.7658257
Product B00001W0KA is recommended for user A1KLRMWW2FWPL4. Score=4.667667
Product B00001WRHJ is recommended for user A1KLRMWW2FWPL4. Score=3.9293647
Product B00004SR8W is recommended for user A1KLRMWW2FWPL4. Score=4.4399595
Product B00004SR8Z is recommended for user A1KLRMWW2FWPL4. Score=4.1355743
Product B00004SR9P is recommended for user A1KLRMWW2FWPL4. Score=3.8792536
Product B00004U1J2 is recommended for user A1KLRMWW2FWPL4. Score=3.7288003


## Сохранение модели
Модель необходимо сохранять после обучения, чтобы была возможность использовать её в дальнейшем или на других платформах. Также эту модель можно будет обучать повторно, что позволит улучшить её качество. Для сохранения необходимо выбрать папку в которую будем сохранять обученную модель. 

In [ ]:
var modelPath = "./Data/MovieRecommenderModel.zip";
public static void SaveModel(MLContext mlContext, DataViewSchema trainDataViewSchema, ITransformer model, string modelPath)
{

    Console.WriteLine("========================== Saving the model to a file ==================================");
    mlContext
        .Model
        .Save(model, trainDataViewSchema, modelPath);
}
SaveModel(mlContext, trainDataView.Schema, model, modelPath);

========================== Saving the model to a file ==================================


## Получение модели из файла
После того как мы сохранили обученную модель в файл её необходимо получить из файла и продолжить работу. Для этого необходимо указать путь до файла, передать схему данных, создать контекст машинного обучения и у него вызвать метод Load, который вернет обученную модель.

In [ ]:
MLContext mLContext = new MLContext();
DataViewSchema schema;
var model1 = mlContext.Model.Load(modelPath, out schema);

Оценим модель и убедимся что она соответствует той модели, что мы сохранили ранее.

In [ ]:
EvaluateModel(mlContext, testDataView, model1);

========================== Evaluating the model =============================
Root Mean Squared Error : 0.3637768376873756
RSquared: 0.8870391363163995


## Выводы
Были изучены методы создания моделей машинного обучения при помощи языка программирования C# и библиотеки ML.NET. В рамках изучения была получена модель машинного обучения, которая имеет процент верных предположений около 90%. Данная модель в дальнейшем будет использована для рекомендации товаров пользователям. ML.NET имеет ряд ограничений для построения больших моделей машинного обучения. Но данные ограничения не существенны, так как работая в одном языке программирования есть возможность использовать общие модели.